Продолжим работу с данными, которые были использованы в ДЗ2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [1]:
from typing import List, Optional, Tuple
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from scipy.stats import ttest_rel

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted
import missingno as msno
import xgboost as xgb
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import (roc_auc_score, roc_curve, auc, accuracy_score, confusion_matrix, \
                             classification_report, plot_confusion_matrix, recall_score, \
                             plot_precision_recall_curve, precision_recall_curve, plot_roc_curve)

In [3]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [8]:
!pip install catboost

     |████████████████████████████████| 76.4 MB 40 kB/s 


In [9]:
import catboost as cb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

warnings.simplefilter("ignore")

## Задание 0. 

__В прошлом ДЗ я использовал LightGBM с обработанными категориальными признаками. В этом дз эти категории нужно обрабатывать согласно заданию преподавателя, поэтому выполнять ДЗ по прошлым лекалам не получится. Сделаем реализацию для Catboost.__

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from google.colab import output
output.enable_custom_widget_manager()

In [12]:
test_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Kaggle_course/data_HW2/assignment_2_test.csv'

In [13]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Kaggle_course/data_HW2/assignment_2_train.csv'

In [14]:
data = pd.read_csv(train_path)

In [15]:
test = pd.read_csv(test_path)

In [16]:
target = data["isFraud"]
data = data.drop(["isFraud"], axis=1)

numerical_features_train = data.select_dtypes(include=np.number)
categorical_features_train = data.select_dtypes(include=["object"])

In [17]:
numerical_features_test = test.select_dtypes(include=np.number)
categorical_features_test = test.select_dtypes(include=["object"])

In [18]:
data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# список признаков с категорией object на трейне
obj_list = categorical_features_train.columns.to_list()
obj_list, len(obj_list)

(['ProductCD',
  'card4',
  'card6',
  'P_emaildomain',
  'R_emaildomain',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9'],
 14)

In [21]:
# переведём пропуски в пустое значение и в строку (для теста и для трейна)
for feature in obj_list:
    data[feature] = categorical_features_train.fillna('')
    data[feature] = categorical_features_train[feature].astype('str')
    test[feature] = categorical_features_test.fillna('')
    test[feature] = categorical_features_test[feature].astype('str')

In [22]:
x_train, x_valid = train_test_split(
    data, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [23]:
y_lead = test['isFraud']
x_lead = test.drop('isFraud', axis=1)

In [24]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [25]:
cb_params = {
    "n_estimators": 1400,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 50,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 35,
    "thread_count": -1,
    "random_seed": 25
}

In [26]:
model_cb = cb.CatBoostClassifier(**cb_params)
model_cb.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6381289	best: 0.6381289 (0)	total: 547ms	remaining: 12m 45s
50:	test: 0.8500685	best: 0.8501336 (47)	total: 23.8s	remaining: 10m 29s
100:	test: 0.8620131	best: 0.8620131 (100)	total: 47.5s	remaining: 10m 10s
150:	test: 0.8685046	best: 0.8685046 (150)	total: 1m 11s	remaining: 9m 50s
200:	test: 0.8758660	best: 0.8758671 (199)	total: 1m 35s	remaining: 9m 27s
250:	test: 0.8822371	best: 0.8822371 (250)	total: 1m 59s	remaining: 9m 6s
300:	test: 0.8849140	best: 0.8849140 (300)	total: 2m 23s	remaining: 8m 42s
350:	test: 0.8876846	best: 0.8876846 (350)	total: 2m 47s	remaining: 8m 19s
400:	test: 0.8890385	best: 0.8890388 (399)	total: 3m 10s	remaining: 7m 53s
450:	test: 0.8902269	best: 0.8902269 (450)	total: 3m 33s	remaining: 7m 29s
500:	test: 0.8907953	best: 0.8908266 (491)	total: 3m 56s	remaining: 7m 3s
550:	test: 0.8912281	best: 0.8912315 (549)	total: 4m 18s	remaining: 6m 38s
600:	test: 0.8918291	best: 0.8918291 (600)	total: 4m 42s	remaining: 6m 15s
650:	test: 0.8921536	best: 0.8921

In [27]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb.predict_proba(x_lead)[:, 1]), 3)}")

ROC AUC train = 0.905
ROC AUC valid = 0.9
ROC AUC test = 0.892
ROC AUC leaderboard = 0.861


## Задание 1.

In [28]:
#признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, 
#прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [29]:
data['TransactionDT'] = pd.to_datetime(data["TransactionDT"], unit='s', origin='2017-12-01')

In [30]:
test['TransactionDT'] = pd.DataFrame(pd.to_datetime(x_lead["TransactionDT"], unit='s', origin='2017-12-01'))

In [31]:
data["year"] = data['TransactionDT'].dt.year
data["month"] = data['TransactionDT'].dt.month
data["day"] = data['TransactionDT'].dt.day
data["hour"] = data['TransactionDT'].dt.hour
data["day_of_week"] = data['TransactionDT'].dt.weekday

In [32]:
test["year"] = test['TransactionDT'].dt.year
test["month"] = test['TransactionDT'].dt.month
test["day"] = test['TransactionDT'].dt.day
test["hour"] = test['TransactionDT'].dt.hour
test["day_of_week"] = test['TransactionDT'].dt.weekday

In [33]:
test

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,hour,day_of_week
0,3287000,1,2018-02-24 19:43:58,226.000,W,12473,555.0,150.0,visa,226.0,credit,299.0,87.0,116.0,NaN,aol.com,nan,2.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,3.0,2.0,6.0,2.0,4.0,4.0,0.0,4.0,3.0,NaN,NaN,NaN,NaN,...,1.0,452.000000,2924.000000,2924.000000,0.0000,412.00000,0.0000,412.0000,206.0000,412.0000,412.0000,452.0,2512.0,2512.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5
1,3287001,0,2018-02-24 19:44:14,3072.000,W,15651,417.0,150.0,visa,226.0,debit,330.0,87.0,NaN,NaN,yahoo.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,3059.949951,3059.949951,3059.949951,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,3059.949951,3059.949951,3059.949951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5
2,3287002,0,2018-02-24 19:44:41,319.950,W,13844,583.0,150.0,visa,226.0,credit,126.0,87.0,9.0,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,319.950012,319.950012,319.950012,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,319.950012,319.950012,319.950012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5
3,3287003,0,2018-02-24 19:45:11,171.000,W,11556,309.0,150.0,visa,226.0,debit,181.0,87.0,3.0,NaN,anonymous.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,6.0,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,...,1.0,0.000000,59.000000,59.000000,0.0000,59.00000,0.0000,59.0000,59.0000,59.0000,59.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5
4,3287004,0,2018-02-24 19:45:12,107.950,W,10985,555.0,150.0,visa,226.0,debit,231.0,87.0,0.0,NaN,anonymous.com,nan,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,10.0,2.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.000000,0.000000,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,3386996,0,2018-03-27 19:12:08,368.990,W,13964,496.0,150.0,mastercard,224.0,debit,299.0,87.0,NaN,NaN,yahoo.com,nan,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,12.0,2.0,154.0,154.0,0.0,553.0,0.0,NaN,NaN,NaN,NaN,...,1.0,368.989990,725.979980,368.989990,0.0000,356.98999,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,368.989990,368.989990,368.989990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,3,27,19,1
99997,3386997,0,2018-03-27 19:12:13,445.330,W,10616,583.0,150.0,visa,226.0,credit,472.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.000000,0.000000,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,3,27,19,1
99998,3386998,0,2018-03-27 19:12:24,15.226,C,9803,583.0,150.0,visa,226.0,credit,NaN,NaN,NaN,NaN,gmail.com,gmail.com,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.791664,0.791666,...,1.0,15.22580

In [34]:
x_train, x_valid = train_test_split(
    data, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [35]:
y_lead = test['isFraud']
x_lead = test.drop('isFraud', axis=1)

In [36]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [37]:
model_cb_data = cb.CatBoostClassifier(**cb_params)
model_cb_data.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6194029	best: 0.6194029 (0)	total: 510ms	remaining: 11m 52s
50:	test: 0.8471564	best: 0.8471564 (50)	total: 23.7s	remaining: 10m 27s
100:	test: 0.8575449	best: 0.8576399 (99)	total: 46.8s	remaining: 10m 1s
150:	test: 0.8689226	best: 0.8689226 (150)	total: 1m 10s	remaining: 9m 45s
200:	test: 0.8754498	best: 0.8754498 (200)	total: 1m 35s	remaining: 9m 27s
250:	test: 0.8826162	best: 0.8826162 (250)	total: 1m 59s	remaining: 9m 6s
300:	test: 0.8861972	best: 0.8861972 (300)	total: 2m 23s	remaining: 8m 43s
350:	test: 0.8882596	best: 0.8882596 (350)	total: 2m 48s	remaining: 8m 24s
400:	test: 0.8892191	best: 0.8892191 (400)	total: 3m 11s	remaining: 7m 58s
450:	test: 0.8904553	best: 0.8904553 (450)	total: 3m 35s	remaining: 7m 34s
500:	test: 0.8912573	best: 0.8912650 (498)	total: 3m 59s	remaining: 7m 10s
550:	test: 0.8914362	best: 0.8915218 (536)	total: 4m 21s	remaining: 6m 43s
600:	test: 0.8917100	best: 0.8917253 (594)	total: 4m 44s	remaining: 6m 18s
Stopped by overfitting detector  (

In [38]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_data.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_data.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_data.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_data.predict_proba(x_lead)[:, 1]), 3)}")

ROC AUC train = 0.906
ROC AUC valid = 0.9
ROC AUC test = 0.892
ROC AUC leaderboard = 0.86


In [39]:
#Task0:
#ROC AUC train = 0.905
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.861
#Task1:
#ROC AUC train = 0.906
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.86
#вывод: преобразование datetime не увеличило скор.

## Задание 2. 

In [40]:
#сделать конкатенацию признаков
# card1 + card2;
# card1 + card2 + card_3 + card_5;
# card1 + card2 + card_3 + card_5 + addr1 + addr2
#Рассматривать их как категориальных признаки.

In [41]:
data['card1_card2'] = data['card1'] + data['card2']

In [42]:
data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,hour,day_of_week,card1_card2
0,2987000,2017-12-02 00:00:00,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,nan,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,...,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,NaN
1,2987001,2017-12-02 00:00:01,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,3159.0
2,2987002,2017-12-02 00:01:09,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,5153.0
3,2987003,2017-12-02 00:01:39,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,nan,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,...,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,18699.0
4,2987004,2017-12-02 00:01:46,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,12,2,0,5,5011.0


In [43]:
data['card1_card2_card3_card5'] = data['card1'] + data['card2'] + data['card3'] + data['card5']

In [44]:
data['card1_card2_card3_card5_addr1_addr2'] = data['card1'] + data['card2'] + data['card3'] + data['card5'] + data['addr1'] + data['addr2']

In [45]:
test['card1_card2'] = test['card1'] + test['card2']

In [46]:
test['card1_card2_card3_card5'] = test['card1'] + test['card2'] + test['card3'] + test['card5']

In [47]:
test['card1_card2_card3_card5_addr1_addr2'] = test['card1'] + test['card2'] + test['card3'] + test['card5'] + test['addr1'] + test['addr2']

In [48]:
task2_feat = ['card1_card2', 'card1_card2_card3_card5', 'card1_card2_card3_card5_addr1_addr2']

In [56]:
for features in task2_feat:
  obj_list.append(features)

In [49]:
categorical_features_train = data.select_dtypes(include=["object"])

In [50]:
categorical_features_test = test.select_dtypes(include=["object"])

In [51]:
for feat in task2_feat:
  categorical_features_train.loc[:, feat] = data[feat]

In [52]:
for feat in task2_feat:
  categorical_features_test.loc[:, feat] = test[feat]

In [53]:
categorical_features_test

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2
0,W,visa,credit,aol.com,nan,T,T,F,M0,T,F,F,F,T,13028.0,13404.0,13790.0
1,W,visa,debit,yahoo.com,nan,nan,nan,nan,nan,nan,T,nan,nan,nan,16068.0,16444.0,16861.0
2,W,visa,credit,gmail.com,nan,T,T,T,nan,nan,T,F,F,T,14427.0,14803.0,15016.0
3,W,visa,debit,anonymous.com,nan,T,T,T,M0,T,T,F,T,T,11865.0,12241.0,12509.0
4,W,visa,debit,anonymous.com,nan,T,T,T,nan,nan,T,T,T,T,11540.0,11916.0,12234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,W,mastercard,debit,yahoo.com,nan,T,T,T,nan,nan,F,F,F,T,14460.0,14834.0,15220.0
99997,W,visa,credit,gmail.com,nan,nan,nan,nan,nan,nan,T,nan,nan,nan,11199.0,11575.0,12134.0
99998,C,visa,credit,gmail.com,gmail.com,nan,nan,nan,M2,nan,nan,nan,nan,nan,10386.0,10762.0,NaN
99999,C,mastercard,credit,gmail.com,gmail.com,nan,nan,nan,M2,nan,nan,nan,nan,nan,16562.0,16884.0,17228.0


In [57]:
for feature in obj_list:
    data[feature] = categorical_features_train.fillna('')
    data[feature] = categorical_features_train[feature].astype('str')
    test[feature] = categorical_features_test.fillna('')
    test[feature] = categorical_features_test[feature].astype('str')

In [58]:
obj_list

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'card1_card2',
 'card1_card2_card3_card5',
 'card1_card2_card3_card5_addr1_addr2']

In [59]:
x_train, x_valid = train_test_split(
    data, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [60]:
y_lead = test['isFraud']
x_lead = test.drop('isFraud', axis=1)

In [61]:
x_lead

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,hour,day_of_week,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2
0,3287000,2018-02-24 19:43:58,226.000,W,12473,555.0,150.0,visa,226.0,credit,299.0,87.0,116.0,NaN,aol.com,nan,2.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,3.0,2.0,6.0,2.0,4.0,4.0,0.0,4.0,3.0,NaN,NaN,NaN,NaN,4.0,...,2924.000000,0.0000,412.00000,0.0000,412.0000,206.0000,412.0000,412.0000,452.0,2512.0,2512.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5,13028.0,13404.0,13790.0
1,3287001,2018-02-24 19:44:14,3072.000,W,15651,417.0,150.0,visa,226.0,debit,330.0,87.0,NaN,NaN,yahoo.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,3059.949951,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,3059.949951,3059.949951,3059.949951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5,16068.0,16444.0,16861.0
2,3287002,2018-02-24 19:44:41,319.950,W,13844,583.0,150.0,visa,226.0,credit,126.0,87.0,9.0,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,319.950012,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,319.950012,319.950012,319.950012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5,14427.0,14803.0,15016.0
3,3287003,2018-02-24 19:45:11,171.000,W,11556,309.0,150.0,visa,226.0,debit,181.0,87.0,3.0,NaN,anonymous.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,6.0,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,6.0,...,59.000000,0.0000,59.00000,0.0000,59.0000,59.0000,59.0000,59.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5,11865.0,12241.0,12509.0
4,3287004,2018-02-24 19:45:12,107.950,W,10985,555.0,150.0,visa,226.0,debit,231.0,87.0,0.0,NaN,anonymous.com,nan,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,10.0,2.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,483.0,...,0.000000,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,2,24,19,5,11540.0,11916.0,12234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,3386996,2018-03-27 19:12:08,368.990,W,13964,496.0,150.0,mastercard,224.0,debit,299.0,87.0,NaN,NaN,yahoo.com,nan,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,12.0,2.0,154.0,154.0,0.0,553.0,0.0,NaN,NaN,NaN,NaN,0.0,...,368.989990,0.0000,356.98999,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,368.989990,368.989990,368.989990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,3,27,19,1,14460.0,14834.0,15220.0
99997,3386997,2018-03-27 19:12:13,445.330,W,10616,583.0,150.0,visa,226.0,credit,472.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.000000,0.0000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,3,27,19,1,11199.0,11575.0,12134.0
99998,3386998,2018-03-27 19:12:24,15.226,C,9803,583.0,150.0,visa,226.0,credit,NaN,NaN,NaN,NaN,gmail.com,gmail.com,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.

In [62]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [63]:
model_cb_task2 = cb.CatBoostClassifier(**cb_params)
model_cb_task2.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6599840	best: 0.6599840 (0)	total: 557ms	remaining: 12m 59s
50:	test: 0.8742477	best: 0.8742477 (50)	total: 27s	remaining: 11m 52s
100:	test: 0.9061301	best: 0.9061301 (100)	total: 52.9s	remaining: 11m 20s
150:	test: 0.9170926	best: 0.9170926 (150)	total: 1m 19s	remaining: 10m 55s
200:	test: 0.9221749	best: 0.9221749 (200)	total: 1m 45s	remaining: 10m 30s
250:	test: 0.9278323	best: 0.9278323 (250)	total: 2m 13s	remaining: 10m 12s
300:	test: 0.9314047	best: 0.9314047 (300)	total: 2m 41s	remaining: 9m 50s
350:	test: 0.9332694	best: 0.9332694 (350)	total: 3m 10s	remaining: 9m 30s
400:	test: 0.9338972	best: 0.9338972 (400)	total: 3m 38s	remaining: 9m 5s
450:	test: 0.9344066	best: 0.9344108 (428)	total: 4m 5s	remaining: 8m 36s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.934410837
bestIteration = 428

Shrink model to first 429 iterations.


In [64]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_task2.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_task2.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_task2.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_task2.predict_proba(x_lead)[:, 1]), 3)}")

ROC AUC train = 0.968
ROC AUC valid = 0.933
ROC AUC test = 0.934
ROC AUC leaderboard = 0.862


In [ ]:
#Task0:
#ROC AUC train = 0.905
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.861
#Task1:
#ROC AUC train = 0.906
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.86
#Task2:
#ROC AUC train = 0.968
#ROC AUC valid = 0.933
#ROC AUC test = 0.934
#ROC AUC leaderboard = 0.862

#вывод: на тесте, валиде, трейне показатели улучшились, а вот на лидерборде ничего не поменялось. 

## Задача 3. 

In [65]:
#Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [66]:
def freq_encoder(data, features):
    data = data.copy()
    for feature in features:
        freq_encoder = data[feature].value_counts(normalize=True)
        data[f"{feature}_freq"] = data[feature].map(freq_encoder)

    return data

In [67]:
features_freq = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']

In [68]:
train_freq = freq_encoder(data, features_freq)

In [69]:
test_freq = freq_encoder(x_lead, features_freq)

In [71]:
train_freq

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,hour,day_of_week,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2,card1_freq,card2_freq,card3_freq,card4_freq,card5_freq,card6_freq,addr1_freq,addr2_freq
0,2987000,2017-12-02 00:00:00,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,nan,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,...,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,nan,nan,nan,0.000061,NaN,0.879737,0.013211,0.000274,0.317939,0.042773,0.982344
1,2987001,2017-12-02 00:00:01,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,3159.0,3411.0,3823.0,0.001244,0.006855,0.879737,0.302783,0.054723,0.317939,0.080004,0.982344
2,2987002,2017-12-02 00:01:09,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,5153.0,5469.0,5886.0,0.001428,0.061413,0.879737,0.657194,0.080269,0.681922,0.046205,0.982344
3,2987003,2017-12-02 00:01:39,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,nan,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,...,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5,18699.0,18966.0,19529.0,0.006600,0.010012,0.879737,0.302783,0.037303,0.681922,0.017276,0.982344
4,2987004,2017-12-02 00:01:46,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,nan,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,12,2,0,5,5011.0,5263.0,5770.0,0.000011,0.030836,0.879737,0.302783,0.054723,0.317939,0.007810,0.982344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,3166995,2018-01-15 19:30:17,39.00,W,1877,310.0,150.0,mastercard,224.0,debit,272.0,87.0,12.0,NaN,gmail.com,nan,2.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,2.0,0.0,5.0,2.0,20.0,20.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,20.0,...,0.0,474.5,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,15,19,0,2187.0,2561.0,2920.0,0.000139,0.004927,0.879737,0.302783,0.128759,0.681922,0.036402,0.982344
179996,3166996,2018-01-15 19:30:37,59.95,W,10075,514.0,150.0,mastercard,224.0,debit,315.0,87.0,NaN,NaN,yahoo.com,nan,4.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,16.0,4.0,239.0,239.0,28.0,365.0,11.0,NaN,NaN,NaN,NaN,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,1,15,19,0,10589.0,10963.0,11365.0,0.000483,0.030836,0.879737,0.302783,0.128759,0.681922,0.042773,0.982344
179997,3166997,2018-01-15 19:30:41,34.00,W,6053,122.0,150.0,mastercard,195.0,debit,330.0,87.0,NaN,NaN,gmail.com,nan,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,28.0,28.0,NaN,NaN,NaN

In [72]:
x_train, x_valid = train_test_split(
    train_freq, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [73]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [74]:
model_cb_task3 = cb.CatBoostClassifier(**cb_params)
model_cb_task3.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6155173	best: 0.6155173 (0)	total: 516ms	remaining: 12m 1s
50:	test: 0.8644827	best: 0.8644827 (50)	total: 27.3s	remaining: 12m 1s
100:	test: 0.9039979	best: 0.9039979 (100)	total: 53.2s	remaining: 11m 24s
150:	test: 0.9158233	best: 0.9158233 (150)	total: 1m 19s	remaining: 10m 59s
200:	test: 0.9235934	best: 0.9235934 (200)	total: 1m 47s	remaining: 10m 38s
250:	test: 0.9270714	best: 0.9270714 (250)	total: 2m 14s	remaining: 10m 16s
300:	test: 0.9317250	best: 0.9317250 (300)	total: 2m 42s	remaining: 9m 53s
350:	test: 0.9336248	best: 0.9336248 (350)	total: 3m 10s	remaining: 9m 29s
400:	test: 0.9355337	best: 0.9355347 (398)	total: 3m 38s	remaining: 9m 4s
450:	test: 0.9363181	best: 0.9363181 (450)	total: 4m 6s	remaining: 8m 38s
500:	test: 0.9377660	best: 0.9377661 (499)	total: 4m 34s	remaining: 8m 12s
550:	test: 0.9378935	best: 0.9378948 (542)	total: 5m 1s	remaining: 7m 44s
600:	test: 0.9379432	best: 0.9379432 (598)	total: 5m 26s	remaining: 7m 14s
650:	test: 0.9380506	best: 0.9380

In [75]:
y_lead = test['isFraud']


In [90]:
train_freq[obj_list]

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2
0,W,discover,credit,nan,nan,T,T,T,M2,F,T,nan,nan,nan,nan,nan,nan
1,W,mastercard,credit,gmail.com,nan,nan,nan,nan,M0,T,T,nan,nan,nan,3159.0,3411.0,3823.0
2,W,visa,debit,outlook.com,nan,T,T,T,M0,F,F,F,F,F,5153.0,5469.0,5886.0
3,W,mastercard,debit,yahoo.com,nan,nan,nan,nan,M0,T,F,nan,nan,nan,18699.0,18966.0,19529.0
4,H,mastercard,credit,gmail.com,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,5011.0,5263.0,5770.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,W,mastercard,debit,gmail.com,nan,T,T,F,M1,T,F,nan,nan,nan,2187.0,2561.0,2920.0
179996,W,mastercard,debit,yahoo.com,nan,nan,nan,nan,nan,nan,T,nan,nan,nan,10589.0,10963.0,11365.0
179997,W,mastercard,debit,gmail.com,nan,T,T,T,M0,T,F,nan,nan,nan,6175.0,6520.0,6937.0
179998,W,visa,debit,nan,nan,nan,nan,nan,M1,T,T,nan,nan,nan,8281.0,8657.0,9016.0


In [77]:
for feature in obj_list:
    test_freq[feature] = categorical_features_test.fillna('')
    test_freq[feature] = categorical_features_test[feature].astype('str')

In [78]:
features_freq = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']

In [79]:
for feature in features_freq:
    test_freq[feature] = test_freq[feature].fillna('')

In [80]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_task3.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_task3.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_task3.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_task3.predict_proba(test_freq)[:, 1]), 3)}")

ROC AUC train = 0.976
ROC AUC valid = 0.938
ROC AUC test = 0.938
ROC AUC leaderboard = 0.868


In [ ]:
#Task0:
#ROC AUC train = 0.905
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.861
#Task1:
#ROC AUC train = 0.906
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.86
#Task2:
#ROC AUC train = 0.968
#ROC AUC valid = 0.933
#ROC AUC test = 0.934
#ROC AUC leaderboard = 0.862
#Task3:
#ROC AUC train = 0.976
#ROC AUC valid = 0.938
#ROC AUC test = 0.938
#ROC AUC leaderboard = 0.868
#Вывод: дополнительно добавилось 0.008 к предыдущему скору на лидерборде.  

## Задание 4. 

In [81]:
#Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt,
# сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [92]:
features_agg = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', \
                'card1_card2', 'card1_card2_card3_card5', 'card1_card2_card3_card5_addr1_addr2']

for feature in features_agg:
    train_freq[f'{feature}_TrAmt_stat'] = train_freq.groupby(feature)['TransactionAmt'].transform('mean') / \
                                        train_freq.groupby(feature)['TransactionAmt'].transform('std')

In [93]:
for feature in features_agg:
    test_freq[f'{feature}_TrAmt_stat'] = test_freq.groupby(feature)['TransactionAmt'].transform('mean') / \
                                        test_freq.groupby(feature)['TransactionAmt'].transform('std')

In [94]:
x_train, x_valid = train_test_split(
    train_freq, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [95]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [96]:
model_cb_task4 = cb.CatBoostClassifier(**cb_params)
model_cb_task4.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.7308786	best: 0.7308786 (0)	total: 568ms	remaining: 13m 14s
50:	test: 0.8733659	best: 0.8733659 (50)	total: 26.9s	remaining: 11m 52s
100:	test: 0.9149772	best: 0.9149772 (100)	total: 53.6s	remaining: 11m 29s
150:	test: 0.9195317	best: 0.9195317 (150)	total: 1m 20s	remaining: 11m 4s
200:	test: 0.9269338	best: 0.9269338 (200)	total: 1m 48s	remaining: 10m 46s
250:	test: 0.9317295	best: 0.9317295 (250)	total: 2m 17s	remaining: 10m 28s
300:	test: 0.9347494	best: 0.9347538 (299)	total: 2m 45s	remaining: 10m 3s
350:	test: 0.9360949	best: 0.9361307 (349)	total: 3m 13s	remaining: 9m 37s
400:	test: 0.9373636	best: 0.9373636 (400)	total: 3m 40s	remaining: 9m 9s
450:	test: 0.9381532	best: 0.9381533 (449)	total: 4m 7s	remaining: 8m 41s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.9381578029
bestIteration = 464

Shrink model to first 465 iterations.


In [97]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_task4.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_task4.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_task4.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_task4.predict_proba(test_freq)[:, 1]), 3)}")

ROC AUC train = 0.973
ROC AUC valid = 0.935
ROC AUC test = 0.938
ROC AUC leaderboard = 0.865


In [99]:
#Task0:
#ROC AUC train = 0.905
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.861
#Task1:
#ROC AUC train = 0.906
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.86
#Task2:
#ROC AUC train = 0.968
#ROC AUC valid = 0.933
#ROC AUC test = 0.934
#ROC AUC leaderboard = 0.862
#Task3:
#ROC AUC train = 0.976
#ROC AUC valid = 0.938
#ROC AUC test = 0.938
#ROC AUC leaderboard = 0.868
#Task4:
#ROC AUC train = 0.973
#ROC AUC valid = 0.935
#ROC AUC test = 0.938
#ROC AUC leaderboard = 0.865
#Вывод: Добавка в виде статистик TransactionAmt скор не улучшила по сравнению с Task3 (я бы откатил назад)

## Задание 5. 

In [ ]:
#Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике.
# Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2,
# и по признакам, созданным в задании 2.

In [100]:
for feature in features_agg:
    train_freq[f'{feature}_D15_stat'] = train_freq.groupby(feature)['D15'].transform('mean') / \
                                        train_freq.groupby(feature)['D15'].transform('std')

In [101]:
for feature in features_agg:
    test_freq[f'{feature}_D15_stat'] = test_freq.groupby(feature)['D15'].transform('mean') / \
                                        test_freq.groupby(feature)['D15'].transform('std')

In [102]:
x_train, x_valid = train_test_split(
    train_freq, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [103]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [104]:
model_cb_task5 = cb.CatBoostClassifier(**cb_params)
model_cb_task5.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.4995238	best: 0.4995238 (0)	total: 298ms	remaining: 6m 56s
50:	test: 0.8737422	best: 0.8737422 (50)	total: 29.9s	remaining: 13m 10s
100:	test: 0.9052134	best: 0.9052134 (100)	total: 56.8s	remaining: 12m 10s
150:	test: 0.9162108	best: 0.9162108 (150)	total: 1m 24s	remaining: 11m 36s
200:	test: 0.9221105	best: 0.9221105 (200)	total: 1m 51s	remaining: 11m 6s
250:	test: 0.9260684	best: 0.9260684 (250)	total: 2m 19s	remaining: 10m 39s
300:	test: 0.9293829	best: 0.9293829 (300)	total: 2m 48s	remaining: 10m 13s
350:	test: 0.9312946	best: 0.9312952 (349)	total: 3m 16s	remaining: 9m 46s
400:	test: 0.9325679	best: 0.9325679 (400)	total: 3m 44s	remaining: 9m 19s
450:	test: 0.9338106	best: 0.9338106 (450)	total: 4m 12s	remaining: 8m 50s
500:	test: 0.9345946	best: 0.9345946 (499)	total: 4m 39s	remaining: 8m 21s
550:	test: 0.9347375	best: 0.9347375 (550)	total: 5m 5s	remaining: 7m 51s
600:	test: 0.9347361	best: 0.9347382 (575)	total: 5m 31s	remaining: 7m 20s
Stopped by overfitting detecto

In [105]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_task5.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_task5.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_task5.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_task5.predict_proba(test_freq)[:, 1]), 3)}")

ROC AUC train = 0.971
ROC AUC valid = 0.934
ROC AUC test = 0.935
ROC AUC leaderboard = 0.865


In [ ]:
#Task0:
#ROC AUC train = 0.905
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.861
#Task1:
#ROC AUC train = 0.906
#ROC AUC valid = 0.9
#ROC AUC test = 0.892
#ROC AUC leaderboard = 0.86
#Task2:
#ROC AUC train = 0.968
#ROC AUC valid = 0.933
#ROC AUC test = 0.934
#ROC AUC leaderboard = 0.862
#Task3:
#ROC AUC train = 0.976
#ROC AUC valid = 0.938
#ROC AUC test = 0.938
#ROC AUC leaderboard = 0.868
#Task4:
#ROC AUC train = 0.973
#ROC AUC valid = 0.935
#ROC AUC test = 0.938
#ROC AUC leaderboard = 0.865
#Task5: 
#ROC AUC train = 0.971
#ROC AUC valid = 0.934
#ROC AUC test = 0.935
#ROC AUC leaderboard = 0.865
#Вывод: Добавка в виде статистик D15 скор не улучшила по сравнению с Task4 (я бы откатил назад)

## Задание 6:

In [ ]:
# выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака.
# После создать отдельных признак - логарифм от TransactionAmt

In [106]:
train_freq['TransactionAmt_int_part'] = divmod(train_freq['TransactionAmt'], 1)[0]

train_freq['TransactionAmt_dec_part'] = divmod(train_freq['TransactionAmt'], 1)[1]

train_freq['TransactionAmt_log'] = np.log(train_freq['TransactionAmt'])

In [107]:
test_freq['TransactionAmt_int_part'] = divmod(test_freq['TransactionAmt'], 1)[0]

test_freq['TransactionAmt_dec_part'] = divmod(test_freq['TransactionAmt'], 1)[1]

test_freq['TransactionAmt_log'] = np.log(test_freq['TransactionAmt'])

In [108]:
x_train, x_valid = train_test_split(
    train_freq, train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.7, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [109]:
train_pool = cb.Pool(x_train, y_train, cat_features=obj_list)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=obj_list)

In [110]:
model_cb_task6 = cb.CatBoostClassifier(**cb_params)
model_cb_task6.fit(train_pool, eval_set=valid_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6023037	best: 0.6023037 (0)	total: 556ms	remaining: 12m 58s
50:	test: 0.8565465	best: 0.8565465 (50)	total: 26.8s	remaining: 11m 49s
100:	test: 0.9040749	best: 0.9040749 (100)	total: 53s	remaining: 11m 21s
150:	test: 0.9118368	best: 0.9118368 (150)	total: 1m 19s	remaining: 10m 55s
200:	test: 0.9203459	best: 0.9203459 (200)	total: 1m 46s	remaining: 10m 34s
250:	test: 0.9257243	best: 0.9257243 (250)	total: 2m 15s	remaining: 10m 19s
300:	test: 0.9323675	best: 0.9323675 (300)	total: 2m 43s	remaining: 9m 58s
350:	test: 0.9343707	best: 0.9343707 (350)	total: 3m 13s	remaining: 9m 37s
400:	test: 0.9355105	best: 0.9355301 (398)	total: 3m 40s	remaining: 9m 10s
450:	test: 0.9356687	best: 0.9356692 (437)	total: 4m 7s	remaining: 8m 41s
500:	test: 0.9357183	best: 0.9357183 (500)	total: 4m 33s	remaining: 8m 10s
550:	test: 0.9359309	best: 0.9359320 (532)	total: 4m 59s	remaining: 7m 41s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.9359319802
bestIteration = 532

Shrink

In [111]:
print(f"ROC AUC train = {round(roc_auc_score(y_train, model_cb_task6.predict_proba(x_train)[:, 1]), 3)}")
print(f"ROC AUC valid = {round(roc_auc_score(y_test, model_cb_task6.predict_proba(x_test)[:, 1]), 3)}")
print(f"ROC AUC test = {round(roc_auc_score(y_valid, model_cb_task6.predict_proba(x_valid)[:, 1]), 3)}")
print(f"ROC AUC leaderboard = " +
    f"{round(roc_auc_score(y_lead, model_cb_task6.predict_proba(test_freq)[:, 1]), 3)}")

ROC AUC train = 0.974
ROC AUC valid = 0.935
ROC AUC test = 0.936
ROC AUC leaderboard = 0.865


In [ ]:
#Скор не поменялся по сравнению с task5.